<center>

<img src="https://www.itv.org/wp-content/uploads/2021/10/logo-itv.svg" width="500" height="200" />

</center>

# <center> **Especialização em Automação** </center>


## <center> **Inteligência Computacional** </center>

---
### <center> **Redes Neurais - Exemplo LSTM** </center>

### <center> Professor: André Almeida Santos</center>

---

# Problema de regressão para previsão temporal com LSTM.

Os dados do exemplo consistem em totais mensais de passageiros aéreos de janeiro de 1949 a dezembro de 1960 de um dataset bem famoso e disponível online [aqui](http://people.se.cmich.edu/lee1c/spss/prj_airlinepassengers.htm).

In [ ]:
# Vamos começar com os imports necessários
import tensorflow as tf
from tensorflow import keras

from keras.layers import Dense, Dropout, LSTM

from sklearn.preprocessing import MinMaxScaler

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

seed = 10
np.random.seed(seed)
tf.random.set_seed(seed)

## Carregando os dados

In [ ]:
dados = pd.read_csv('https://query.data.world/s/bdhpzucmfzowx44mhnipyfd2qv7ufu')

dados = dados.set_index("Month") # define como indice a coluna "Month"

dados.index = pd.to_datetime(dados.index, format = '%Y-%m') # converte a coluna Time (df.index) para formato datetime

In [ ]:
dados

In [ ]:
# visualização dos dados

plt.rcParams['figure.figsize'] = 19, 6 # ajusta as dimensões do grafico
plt.rcParams.update({'font.size': 20}) # ajusta tamanho da fonte
plt.plot(dados, color = 'blue', marker = "o")
plt.title("Passageiros Internacionais Transportados", fontsize = 22)
plt.ylabel("Passageiros")
plt.xlabel("Data")
plt.show()

## Pre processando os dados

In [ ]:
tamanho_dados_treino = int(len(dados) * 0.75)  # cria o tamanho dos dados inteiros 

dados_treino = dados[0:tamanho_dados_treino] # separa os dados de treino para criar o modelo
dados_teste = dados[tamanho_dados_treino:len(dados)] # separa os dados para testar o modelo

# mostra como ficará a divisão dos dados 
plt.rcParams['figure.figsize'] = 19, 6
plt.rcParams.update({'font.size': 20})
plt.plot(dados_treino, color = 'blue', marker = "o")
plt.plot(dados_teste, color = 'red', marker = "o")
plt.grid(True)
plt.show()

In [ ]:
dados_treino

Decompondo a serie com a biblioteca statsmodels

In [ ]:
# modelo de decomposição da série
from statsmodels.tsa.seasonal import seasonal_decompose

result = seasonal_decompose(dados_treino, model = 'multiplicative') 
result.plot()
plt.show()

### Análise da importância dos dados passados

Vamos usar o **RandomForestRegressor** para determinar os lags mais importantes na previsão atual dentro do último ano.

In [ ]:
dataframe = pd.DataFrame()

for i in range(12, 0, -1):
	dataframe['t-'+str(i)] = dados_treino.shift(i)
dataframe['t'] = dados_treino.values

dataframe.head(20)

Agora preciso considerar o seguinte:

dados de entrada -> X = t-12,	t-11,	t-10,	t-9,	t-8,	t-7, t-6, t-5, t-4, t-3,	t-2,	t-1

saída -> y = t

Quero predizer t em função dos 13 tempos passados com uma random forest.

Como a árvore de decisão separa os dados em função da melhor divisão, coseguimos estimar os melhores features para usar na predição com a rede neural.


In [ ]:
from sklearn.ensemble import RandomForestRegressor

# divide entre entradas e saida para o random forest
X = dataframe.iloc[12:, 0:-1]
y = dataframe.iloc[12:, -1]

# fit random forest model
model = RandomForestRegressor(n_estimators=500, random_state=1)
model.fit(X, y)

# mostrar pontuação de importância
print(model.feature_importances_)

# plot pontuação de importância
names = dataframe.columns.values[0:-1]

ticks = [i for i in range(len(names))]
plt.bar(ticks, model.feature_importances_)
plt.xticks(ticks, names)
plt.show()

### Conhecendo os dados mais importantes, vamos reorganizar nosso dataset conforme as análises.

In [ ]:
dados_treino_input = dataframe.loc[:, ['t-12', 't-1']][12:]

dados_treino_target = dataframe.loc[:, 't'][12:]

In [ ]:
dados_treino_input

In [ ]:
dados_treino_target

### Normalização dos dados

In [ ]:
#normalizando os dados treino de entrada
norm_dados_treino = MinMaxScaler(feature_range = (0, 1))

dados_treino_input = norm_dados_treino.fit_transform(dados_treino_input)

In [ ]:
# normalizando os dados treino target

# primeiro vou colocar os dados no formato array aceito pelo método abaixo.
dados_treino_target = np.array(dados_treino_target).reshape(-1,1)

norm_dados_treino_target = MinMaxScaler(feature_range = (0, 1))

dados_treino_target = norm_dados_treino_target.fit_transform(dados_treino_target)

In [ ]:
dados_treino_input

In [ ]:
# prepara para entrada da rede

#dados_treino_input = np.array(dados_treino_input)
dados_treino_input = np.reshape(dados_treino_input, (dados_treino_input.shape[0], dados_treino_input.shape[1], 1))

dados_treino_input

In [ ]:
dados_treino_input.shape[:]

## Construindo a rede neural com Tensorflow e keras



Definindo o input

In [ ]:
inputs = keras.Input(shape=dados_treino_input.shape[1:], name='entrada')

In [ ]:
# inputs: A 3D tensor with shape [batch, timesteps, feature]

inputs.shape

Definindo as camadas intermediárias

In [ ]:
lstm_int = LSTM(units = 18, return_sequences = True, name='lstm_int_1')(inputs)

In [ ]:
lstm_int2 = LSTM(units = 15, return_sequences = True, name='lstm_int_2')(lstm_int)

In [ ]:
lstm_int3 = LSTM(units = 10, return_sequences = True, name='lstm_int_3')(lstm_int2)

In [ ]:
drop = Dropout(0.2)(lstm_int3)

In [ ]:
lstm_int3 = LSTM(units = 10, name='lstm_3')(drop)

In [ ]:
drop2 = Dropout(0.2)(lstm_int3)

Definindo output

In [ ]:
output = Dense(units = 1, activation = 'linear', name='saida')(drop2)

Construindo o objeto da rede

In [ ]:
modelo = keras.Model(inputs=inputs, outputs=output, name="modelo_LSTM")

In [ ]:
modelo.summary()

In [ ]:
keras.utils.plot_model(modelo, "meu_primeiro_modelo_LSTM_com_informação_do_shape.png", show_shapes=True)

In [ ]:
modelo.compile(
    loss='mse',
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
    metrics=[[tf.keras.metrics.RootMeanSquaredError()]],
)

## Treinando o modelo

In [ ]:
treino = modelo.fit(dados_treino_input, dados_treino_target, batch_size=8, epochs=500)

In [ ]:
# plot dos dados de treino

plt.title('Erro do modelo durante treino')
plt.ylabel('mse')
plt.xlabel('Época')
plt.plot(treino.history['loss'], label='Treino')
plt.legend()
plt.show()

In [ ]:
# plot dos dados de treino

plt.title('Erro do modelo durante treino')
plt.ylabel('rmse')
plt.xlabel('Época')
plt.plot(treino.history['root_mean_squared_error'], label='Treino')
plt.legend()
plt.show()

## Prevendo os dados de teste

Aqui é preciso pre processar os dados de teste conforme processo do treino.

Os dados de teste também precisam ser normalizados e formatados conforme os de treino.

In [ ]:
# Vamos normalizar agora

dataframe = pd.DataFrame()

for i in range(12, 0, -1):
	dataframe['t-'+str(i)] = dados_teste.shift(i)
dataframe['t'] = dados_teste.values

dataframe.head(20)

In [ ]:
dados_teste_input = dataframe.loc[:, ['t-12', 't-1']][12:]

dados_teste_target = dataframe.loc[:, 't'][12:]

In [ ]:
dados_teste_target

In [ ]:
norm_dados_teste = MinMaxScaler(feature_range = (0, 1))
dados_teste_input = norm_dados_teste.fit_transform(dados_teste_input)

In [ ]:
dados_teste_target = np.array(dados_teste_target).reshape(-1,1)

norm_dados_teste_target = MinMaxScaler(feature_range = (0, 1))
norm_dados_teste_target.fit(dados_teste_target)

In [ ]:
dados_teste_target

In [ ]:
# prepara para entrada da rede

dados_teste_input = np.reshape(dados_teste_input, (dados_teste_input.shape[0], dados_teste_input.shape[1], 1))

dados_teste_input

In [ ]:
# prevendo

previsoes = modelo.predict(dados_teste_input)

In [ ]:
previsoes

In [ ]:
plt.plot(norm_dados_teste_target.inverse_transform(previsoes), color = 'blue', marker = "o", label= 'Previsão')
plt.plot(dados_teste_target, color = 'red', marker = "o", label='Original')
plt.legend()
plt.grid(True)
plt.show()